In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [ ]:
path = '/content/drive/MyDrive/ML_final/'

In [ ]:
fold0 = pd.read_csv(path+"fold_0_data.txt",sep = "\t")
fold1 = pd.read_csv(path+"fold_1_data.txt",sep = "\t")
fold2 = pd.read_csv(path+"fold_2_data.txt",sep = "\t")
fold3 = pd.read_csv(path+"fold_3_data.txt",sep = "\t")
fold4 = pd.read_csv(path+"fold_4_data.txt",sep = "\t")
total_data = pd.concat([fold0, fold1, fold2, fold3, fold4], ignore_index=True)
print(total_data.shape)
total_data.info()

(13923, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13923 entries, 0 to 13922
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             13923 non-null  object 
 1   original_image      13923 non-null  object 
 2   face_id             13922 non-null  float64
 3   age                 13922 non-null  object 
 4   gender              13814 non-null  object 
 5   x                   13922 non-null  float64
 6   y                   13922 non-null  float64
 7   dx                  13922 non-null  float64
 8   dy                  13922 non-null  float64
 9   tilt_ang            13921 non-null  float64
 10  fiducial_yaw_angle  13921 non-null  float64
 11  fiducial_score      13920 non-null  float64
dtypes: float64(8), object(4)
memory usage: 1.3+ MB


In [ ]:
total_data.head()

,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1.0,"(25, 32)",f,0.0,414.0,1086.0,1383.0,-115.0,30.0,17.0
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2.0,"(25, 32)",m,301.0,105.0,640.0,641.0,0.0,0.0,94.0
2,30601258@N03,10437979845_5985be4b26_o.jpg,1.0,"(25, 32)",f,2395.0,876.0,771.0,771.0,175.0,-30.0,74.0
3,30601258@N03,10437979845_5985be4b26_o.jpg,3.0,"(25, 32)",m,752.0,1255.0,484.0,485.0,180.0,0.0,47.0
4,30601258@N03,11816644924_075c3d8d59_o.jpg,2.0,"(25, 32)",m,175.0,80.0,769.0,768.0,-75.0,0.0,34.0


In [ ]:
imp_data = total_data[['age', 'gender', 'x', 'y', 'dx', 'dy']].copy()
imp_data.info()
img_path = []
for row in total_data.iterrows():
  try:
    p = path+"faces/"+row[1].user_id+"/coarse_tilt_aligned_face."+str(int(row[1].face_id))+"."+row[1].original_image
    img_path.append(p)
  except ValueError:
    p = 'None'
    img_path.append(p)

imp_data['img_path'] = img_path
imp_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13923 entries, 0 to 13922
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     13922 non-null  object 
 1   gender  13814 non-null  object 
 2   x       13922 non-null  float64
 3   y       13922 non-null  float64
 4   dx      13922 non-null  float64
 5   dy      13922 non-null  float64
dtypes: float64(4), object(2)
memory usage: 652.8+ KB


,age,gender,x,y,dx,dy,img_path
0,"(25, 32)",f,0.0,414.0,1086.0,1383.0,/content/drive/MyDrive/ML_final/faces/30601258...
1,"(25, 32)",m,301.0,105.0,640.0,641.0,/content/drive/MyDrive/ML_final/faces/30601258...
2,"(25, 32)",f,2395.0,876.0,771.0,771.0,/content/drive/MyDrive/ML_final/faces/30601258...
3,"(25, 32)",m,752.0,1255.0,484.0,485.0,/content/drive/MyDrive/ML_final/faces/30601258...
4,"(25, 32)",m,175.0,80.0,769.0,768.0,/content/drive/MyDrive/ML_final/faces/30601258...


In [ ]:
age_mapping = [('(0, 2)', '0-2'), ('2', '0-2'), ('3', '0-2'), ('(4, 6)', '4-6'), ('(8, 12)', '8-13'), ('13', '8-13'), ('22', '15-20'), ('(8, 23)','15-20'), ('23', '25-32'), ('(15, 20)', '15-20'), ('(25, 32)', '25-32'), ('(27, 32)', '25-32'), ('32', '25-32'), ('34', '25-32'), ('29', '25-32'), ('(38, 42)', '38-43'), ('35', '38-43'), ('36', '38-43'), ('42', '48-53'), ('45', '38-43'), ('(38, 43)', '38-43'), ('(38, 42)', '38-43'), ('(38, 48)', '48-53'), ('46', '48-53'), ('(48, 53)', '48-53'), ('55', '48-53'), ('56', '48-53'), ('(60, 100)', '60+'), ('57', '60+'), ('58', '60+')]

age_mapping_dict = {each[0]: each[1] for each in age_mapping}
print(age_mapping_dict)
drop_labels = []
for idx, each in enumerate(imp_data.age):
  if each == 'None':
    drop_labels.append(idx)
  else:
    try:
      imp_data.age.loc[idx] = age_mapping_dict[each]
    except KeyError:
      imp_data.age.loc[idx] = each

imp_data = imp_data.drop(labels=drop_labels, axis=0) #droped None values
imp_data.age.value_counts(dropna=False)

{'(0, 2)': '0-2', '2': '0-2', '3': '0-2', '(4, 6)': '4-6', '(8, 12)': '8-13', '13': '8-13', '22': '15-20', '(8, 23)': '15-20', '23': '25-32', '(15, 20)': '15-20', '(25, 32)': '25-32', '(27, 32)': '25-32', '32': '25-32', '34': '25-32', '29': '25-32', '(38, 42)': '38-43', '35': '38-43', '36': '38-43', '42': '48-53', '45': '38-43', '(38, 43)': '38-43', '(38, 48)': '48-53', '46': '48-53', '(48, 53)': '48-53', '55': '48-53', '56': '48-53', '(60, 100)': '60+', '57': '60+', '58': '60+'}


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


25-32    4027
0-2      2034
38-43    1815
8-13     1756
4-6      1752
15-20    1087
48-53     675
60+       674
NaN         1
Name: age, dtype: int64

In [ ]:
# Unzipping facial-age.zip to datasets folder.

!unzip -uq "/content/drive/MyDrive/ML_final/facial-age.zip" -d "/content/drive/MyDrive/ML_final/Age_datasets"

In [ ]:
# Unzipping facial-age.zip to datasets folder.

!unzip -uq "/content/drive/MyDrive/ML_final/UTKFace.zip" -d "/content/drive/MyDrive/ML_final/Age_datasets"

In [ ]:
imp_data

,age,gender,x,y,dx,dy,img_path
0,25-32,f,0.0,414.0,1086.0,1383.0,/content/drive/MyDrive/ML_final/faces/30601258...
1,25-32,m,301.0,105.0,640.0,641.0,/content/drive/MyDrive/ML_final/faces/30601258...
2,25-32,f,2395.0,876.0,771.0,771.0,/content/drive/MyDrive/ML_final/faces/30601258...
3,25-32,m,752.0,1255.0,484.0,485.0,/content/drive/MyDrive/ML_final/faces/30601258...
4,25-32,m,175.0,80.0,769.0,768.0,/content/drive/MyDrive/ML_final/faces/30601258...
...,...,...,...,...,...,...,...
13918,38-43,f,0.0,6.0,1251.0,1422.0,/content/drive/MyDrive/ML_final/faces/10069023...
13919,8-13,m,683.0,627.0,714.0,714.0,/content/drive/MyDrive/ML_final/faces/33627988...
13920,8-13,f,2231.0,471.0,580.0,580.0,/content/drive/MyDrive/ML_final/faces/33627988...
13921,38-43,m,1480.0,976.0,880.0,880.0,/content/drive/MyDrive/ML_final/faces/33627988...


In [ ]:
imp_data = imp_data.dropna()
clean_data = imp_data[imp_data.gender != 'u'].copy()
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13053 entries, 0 to 13921
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       13053 non-null  object 
 1   gender    13053 non-null  object 
 2   x         13053 non-null  float64
 3   y         13053 non-null  float64
 4   dx        13053 non-null  float64
 5   dy        13053 non-null  float64
 6   img_path  13053 non-null  object 
dtypes: float64(4), object(3)
memory usage: 815.8+ KB


In [ ]:
imp_data

,age,gender,x,y,dx,dy,img_path
0,25-32,f,0.0,414.0,1086.0,1383.0,/content/drive/MyDrive/ML_final/faces/30601258...
1,25-32,m,301.0,105.0,640.0,641.0,/content/drive/MyDrive/ML_final/faces/30601258...
2,25-32,f,2395.0,876.0,771.0,771.0,/content/drive/MyDrive/ML_final/faces/30601258...
3,25-32,m,752.0,1255.0,484.0,485.0,/content/drive/MyDrive/ML_final/faces/30601258...
4,25-32,m,175.0,80.0,769.0,768.0,/content/drive/MyDrive/ML_final/faces/30601258...
...,...,...,...,...,...,...,...
13917,38-43,f,1558.0,272.0,523.0,523.0,/content/drive/MyDrive/ML_final/faces/10069023...
13918,38-43,f,0.0,6.0,1251.0,1422.0,/content/drive/MyDrive/ML_final/faces/10069023...
13919,8-13,m,683.0,627.0,714.0,714.0,/content/drive/MyDrive/ML_final/faces/33627988...
13920,8-13,f,2231.0,471.0,580.0,580.0,/content/drive/MyDrive/ML_final/faces/33627988...


In [ ]:
age_to_label_map = {
    '0-2'  :0,
    '4-6'  :1,
    '8-13' :2,
    '15-20':3,
    '25-32':4,
    '38-43':5,
    '48-53':6,
    '60+'  :7
}

clean_data['age'] = clean_data['age'].apply(lambda age: age_to_label_map[age])
clean_data.head()

NameError: ignored

In [ ]:
import os
yourPath = '/content/drive/MyDrive/ML_final/Age_datasets/facial-age'
allFileList = os.listdir(yourPath)
X_train = []
Y_train = []
for file in allFileList:
  print(int(file))
  p = '/content/drive/MyDrive/ML_final/Age_datasets/facial-age/'+file
  imgpath = os.listdir(p)
  for i in imgpath:
    Y_train.append(int(file))
    filepath = p+'/'+i
    X_train.append(filepath)
    print(filepath)
    

串流輸出內容已截斷至最後 5000 行。
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/6183.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/5129.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/626.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/5647.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/5469.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/5869.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/8873.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/5286.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/8512.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/501.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/7540.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/7217.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/8553.png
/content/drive/MyDrive/ML_final/Age_datasets/facial-age/005/7977.png
/content/drive/

In [ ]:
import numpy as np
np.shape(X_train)

(9778,)

In [ ]:
np.shape(Y_train)

(9778,)

In [ ]:
Y_train

[48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,


In [ ]:
age_labels = {0: (0, 2),
              1: (3, 7),
              2: (8, 14),
              3: (15, 24),
              4: (25, 37),
              5: (38, 47),
              6: (48, 59),
              7: (60, 200)}

In [ ]:
Y_label = []
for j in Y_train:
    for i in age_labels:
        if(j >= age_labels[i][0] and j <= age_labels[i][1]):
            Y_label.append(i)

In [ ]:
labeled_dataset_list = []
labeled_dataset = pd.DataFrame({'age':Y_label,'img':X_train})
# Spilt the dataset by the age label, get the maximal length in the datasets.
for i in range(8):
    data = labeled_dataset.loc[labeled_dataset['age'] == i]
    labeled_dataset_list.append(data)
    print("Label: " + str(i) + ", count: " + str(len(data)) )

Label: 0, count: 1587
Label: 1, count: 964
Label: 2, count: 916
Label: 3, count: 1358
Label: 4, count: 1663
Label: 5, count: 739
Label: 6, count: 1101
Label: 7, count: 1450


In [ ]:
sample_count = 2000

# Re-sample the dataset to the sample_count, and concat the dataset to balanced_data.
for i in range(8):
    labeled_dataset_list[i] = labeled_dataset_list[i].sample(sample_count, replace=True)
balanced_dataset = pd.concat(labeled_dataset_list, axis=0, ignore_index=True)

print(balanced_dataset)

       age                                                img
0        0  /content/drive/MyDrive/ML_final/Age_datasets/f...
1        0  /content/drive/MyDrive/ML_final/Age_datasets/f...
2        0  /content/drive/MyDrive/ML_final/Age_datasets/f...
3        0  /content/drive/MyDrive/ML_final/Age_datasets/f...
4        0  /content/drive/MyDrive/ML_final/Age_datasets/f...
...    ...                                                ...
15995    7  /content/drive/MyDrive/ML_final/Age_datasets/f...
15996    7  /content/drive/MyDrive/ML_final/Age_datasets/f...
15997    7  /content/drive/MyDrive/ML_final/Age_datasets/f...
15998    7  /content/drive/MyDrive/ML_final/Age_datasets/f...
15999    7  /content/drive/MyDrive/ML_final/Age_datasets/f...

[16000 rows x 2 columns]


In [ ]:
import tensorflow as tf
import keras
import PIL
import numpy as np
import pandas as pd
import os
import math
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
X = balanced_dataset['img']
y = balanced_dataset['age']
X =X.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Load train_image
image_list = []

for file_name in X_train:
    image_array = np.array(PIL.Image.open(file_name))
    resized_image_array = cv2.resize(image_array, (128, 128))
    image_list.append(resized_image_array)
train_images = np.array(image_list)

# Load test_image
image_list = []

for file_name in X_test:
    image_array = np.array(PIL.Image.open(file_name))
    resized_image_array = cv2.resize(image_array, (128, 128))
    image_list.append(resized_image_array)
test_images = np.array(image_list)
y_train = y_train.to_numpy(dtype = int)
y_test = y_test.to_numpy(dtype = int)

In [ ]:
train_images.shape

(11200, 200, 200, 3)

In [ ]:
y_train = y_train.reshape((-1,1))

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(128, 128, 3), filters=96, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=8, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 200, 200, 96)      4704      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 100, 100, 96)     0         
 2D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 100, 100, 96)     384       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 100, 100, 96)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 100, 100, 256)     393472    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 50, 50, 256)     

In [ ]:
import tensorflow as tf
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
y_train.shape

(11200, 1)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5) # Callback for earlystopping
model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy" , metrics=['accuracy'])
history = model.fit(train_images, y_train, batch_size=32, epochs=15, validation_data=(test_images, y_test), callbacks=[callback])

Epoch 1/15
350/350 [==============================] - 319s 855ms/step - loss: 1.6664 - accuracy: 0.3880 - val_loss: 1.3700 - val_accuracy: 0.4375
Epoch 2/15
350/350 [==============================] - 312s 891ms/step - loss: 1.0671 - accuracy: 0.5704 - val_loss: 0.9691 - val_accuracy: 0.6181
Epoch 3/15
350/350 [==============================] - 312s 891ms/step - loss: 0.7622 - accuracy: 0.7014 - val_loss: 0.7482 - val_accuracy: 0.7237
Epoch 4/15
350/350 [==============================] - 298s 853ms/step - loss: 0.5213 - accuracy: 0.8044 - val_loss: 0.6391 - val_accuracy: 0.7800
Epoch 5/15
350/350 [==============================] - 299s 853ms/step - loss: 0.3243 - accuracy: 0.8831 - val_loss: 0.5273 - val_accuracy: 0.8396
Epoch 6/15
350/350 [==============================] - 312s 891ms/step - loss: 0.2030 - accuracy: 0.9334 - val_loss: 0.4845 - val_accuracy: 0.8577
Epoch 7/15
350/350 [==============================] - 312s 891ms/step - loss: 0.1357 - accuracy: 0.9546 - val_loss: 0.5447 -

In [ ]:
model.save('/content/drive/MyDrive/ML_final/facial_md.h5')

test_loss, test_acc = model.evaluate(test_images, y_test, verbose=2)
print(test_acc)

150/150 - 28s - loss: 0.5567 - accuracy: 0.8790 - 28s/epoch - 185ms/step
0.8789583444595337


In [ ]:
X = clean_data[['img_path']]
y = clean_data[['age']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('Train data shape {}'.format(X_train.shape))
print('Test data shape {}'.format(X_test.shape))

train_images = []
test_images = []

cnt = 0
for row in X_train.iterrows():
  print(cnt)
  cnt += 1
  try:
    image = Image.open(row[1].img_path)
    image = image.resize((128, 128))   # Resize the image
    data = np.asarray(image)
    train_images.append(data)
  except FileNotFoundError:
    pass

cnt = 0
for row in X_test.iterrows():
  print(cnt)
  cnt += 1
  try:
    image = Image.open(row[1].img_path)
    image = image.resize((128, 128))  # Resize the image
    data = np.asarray(image)
    test_images.append(data)
  except FileNotFoundError:
    pass

train_images = np.asarray(train_images)
test_images = np.asarray(test_images)

print('Train images shape {}'.format(train_images.shape))
print('Test images shape {}'.format(test_images.shape))

Train data shape (9137, 1)
Test data shape (3916, 1)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
2

KeyboardInterrupt: ignored

In [ ]:
print(y_train.shape)

(9137, 1)


In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(128, 128, 3), filters=96, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=8, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 96)      4704      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 64, 64, 96)       384       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 64, 64, 96)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 256)       393472    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 256)     

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5) # Callback for earlystopping
model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy" , metrics=['accuracy'])
history = model.fit(train_images, y_train, batch_size=32, epochs=50, validation_data=(test_images, y_test), callbacks=[callback])

model.save('/content/drive/MyDrive/ML_final/age_model')

test_loss, test_acc = model.evaluate(test_images, y_test, verbose=2)
print(test_acc)

Epoch 1/50
286/286 [==============================] - 110s 384ms/step - loss: 1.9179 - accuracy: 0.3078 - val_loss: 1.9216 - val_accuracy: 0.3057
Epoch 2/50
286/286 [==============================] - 109s 381ms/step - loss: 1.9155 - accuracy: 0.3081 - val_loss: 1.9213 - val_accuracy: 0.3054
Epoch 3/50
286/286 [==============================] - 108s 379ms/step - loss: 1.9159 - accuracy: 0.3081 - val_loss: 1.9214 - val_accuracy: 0.3057
Epoch 4/50
286/286 [==============================] - 108s 379ms/step - loss: 1.9153 - accuracy: 0.3080 - val_loss: 1.9214 - val_accuracy: 0.3057
Epoch 5/50
286/286 [==============================] - 108s 379ms/step - loss: 1.9152 - accuracy: 0.3082 - val_loss: 1.9215 - val_accuracy: 0.3059
Epoch 6/50
286/286 [==============================] - 109s 380ms/step - loss: 1.9155 - accuracy: 0.3082 - val_loss: 1.9218 - val_accuracy: 0.3057
Epoch 7/50
286/286 [==============================] - 109s 381ms/step - loss: 1.9153 - accuracy: 0.3079 - val_loss: 1.9215 -

In [ ]:
from keras import backend
from keras.models import load_model
def h5_to_pb(h5model):
  out_nodes = []
  for i in range(len(h5model.outputs)):
    out_nodes.append("output_" + str(i+1))
    tf.identity(h5model.outputs[i], "output_" + str(i+1))
  sess = backend.get_session()
  from tensorflow.python.framework import graph_util, graph_io
  init_graph = sess.graph.as_graph_def()
  main_graph = graph_util.convert_variables_to_constants(sess, init_graph, out_nodes)
  graph_io.write_graph(main_graph, "/content/drive/MyDrive/ML_final/", name = "age_model.pb", as_text = False)

In [ ]:
h5model = load_model("/content/drive/MyDrive/ML_final/age_model50.h5")
tf.saved_model.save(h5model, export_dir="/content/drive/MyDrive/ML_final/age_model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_final/age_model/assets
